In [ ]:
#18i-0715 PAWAN KUMAR
#18i-0468 FARAZ AHMED

In [1338]:
#importing libraries
import pandas
import csv
import random

In [1339]:
#read courses.csv
courses_name = list()
courses_code = list()
with open('courses.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] not in courses_code:
            courses_code.append(str(row[0]))
            courses_name.append(str(row[1]))

In [1340]:
#read courses.csv
students = list()
with open('studentNames.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if row[0] not in courses_code:
            students.append(str(row[0]))

In [1341]:
#read studentCourse.csv
student_names = list()
student_courses_code = list()

with open('studentCourse.csv', 'r') as file:
    reader = csv.reader(file) 
    for row in reader:
        student_names.append(str(row[1]))
        student_courses_code.append(str(row[2]))

In [1342]:
#read invigilator.csv
invigilator_names = list()

with open('teachers.csv', 'r') as file:
    reader = csv.reader(file) 
    for row in reader:
        invigilator_names.append(str(row[0]))
#print(len(invigilator_names))

In [1343]:
#essential arrays
rooms = ["C301", "C302", "C303", "C304", "C305", "C306", "C307", "C308"]
days = ["Monday1", "Tuesday1", "Wednesday1", "Thursday1", "Friday1", "Saturday1"
        , "Monday2", "Tuesday2", "Wednesday2", "Thursday2", "Friday2", "Saturday2"
       , "Monday3", "Tuesday3", "Wednesday3", "Thursday3", "Friday3", "Saturday3"
       , "Monday4", "Tuesday4", "Wednesday4", "Thursday4", "Friday4", "Saturday4"
       , "Monday5", "Tuesday5", "Wednesday5", "Thursday5", "Friday5", "Saturday5"]
exam_timings = ["AM 9:00 - 12:00", "PM 2:00 - 5:00"]

In [1344]:
# dictionary with key courses to student
student_courses_dict = dict()
for i in range(len(courses_code)):
    if courses_code[i] not in student_courses_dict:
        student_courses_dict[courses_code[i]] = list()
        student_list = list()
        for j in range(len(student_courses_code)-1):
            if str(courses_code[i]) == str(student_courses_code[j]):
                student_list.append(student_names[j])
    student_courses_dict[courses_code[i]].extend(student_list)
            
        
#print(student_courses_dict[courses_code[0]])

In [1345]:
# dictionary with key student to courses
courses_student_dict = dict()
for i in range(len(students)):
    if students[i] not in courses_student_dict:
        courses_student_dict[students[i]] = list()
        courses_list = list()
        for j in range(len(student_names)-1):
            if str(students[i]) == str(student_names[j]):
                courses_list.append(student_courses_code[j])
    courses_student_dict[students[i]].extend(courses_list)
            
        
#print((courses_student_dict[students[199]]))

In [1346]:
class CourseExam:
    def __init__(self, course_code, course_name, room_no, invigilator, students_enrolled, day, time):
        self.course_code = course_code
        self.course_name = course_name
        self.room_no = room_no
        self.invigilator = invigilator
        self.students_enrolled = students_enrolled
        self.day = day
        self.time = time
    

In [1347]:
class Chromosome:
    def __init__(self, time_table, fitness):
        
        self.time_table = create_time_Table(time_table)
        self.fitness = fitness

In [1348]:
#function to create one chromosome
def create_time_Table(time_table):
    for i in range(0, len(courses_code)):
        room_no = random.randint(0,7)
        invigilator = random.randint(0,62)
        day = random.randint(0,29)
        time = random.randint(0,1)
        #print(room_no, invigilator, day, time)
        obj = CourseExam(courses_code[i], courses_name[i], rooms[room_no], invigilator_names[invigilator],
                         student_courses_dict[courses_code[i]], days[day], exam_timings[time])
        time_table.append(obj)
        #print(time_table[0].course_code)
    return time_table
#print(len(time_table))


In [1349]:
def soft_constraint_2(chromosome):
    #each student has one exam at a time
    for key in courses_student_dict:
        exam_day = list()
        exam_time = list()
        #checking each course of key student
        for value in courses_student_dict[key]:
            for i in chromosome.time_table:
                if str(value) == str(i.course_code):
                    exam_time.append(i.time)
                    exam_day.append(i.day)
                    
        for i in range(len(exam_day)-2):
            if exam_day[i] == exam_day[i+1]:
                return False
    return True
        

In [1350]:
def hard_constraint_1(chromosome):
    #if no of courses is equal to courses in chromosome timetable
    if len(chromosome.time_table) == len(courses_code):
        return True
    else:
        return False

In [1351]:
def hard_constraint_2(chromosome):
    #student must be enrolled in less than 3 courses
    for i in chromosome.time_table:
        for j in i.students_enrolled:
            if(len(courses_student_dict[j]) < 3):
                return False
    
    #each student has one exam at a time
    for key in courses_student_dict:
        exam_day = list()
        exam_time = list()
        #checking each course of key student
        for value in courses_student_dict[key]:
            for i in chromosome.time_table:
                if str(value) == str(i.course_code):
                    exam_time.append(i.time)
                    exam_day.append(i.day)
        
        for i in range(len(exam_day)-2):
            if exam_day[i] == exam_day[i+1]:
                if exam_time[i] == exam_time[i+1]:
                    return False
                    
        
    return True

In [1352]:
def hard_constraint_3(chromosome):
    for i in range(len(chromosome.time_table)-1):
        if str(chromosome.time_table[i].day) == "Saturday1" or str(chromosome.time_table[i].day) == "Saturday2" or str(chromosome.time_table[i].day) == "Saturday3" or str(chromosome.time_table[i].day) == "Saturday4" or str(chromosome.time_table[i].day) == "Saturday5":
            #print("F weekend")
            return False

In [1353]:
def hard_constraint_4(chromosome):
    for i in chromosome.time_table:
        if str(i.time) == "AM 3:00 - 6:00":
            #print("F time")
            return False

In [1354]:
def hard_constraint_5(chromosome):
    for i in chromosome.time_table:
        for j in chromosome.time_table:
            if i != j:
                if str(i.invigilator) == str(j.invigilator):
                    if str(i.day) == str(j.day):
                        if str(i.time) == str(j.time):
                            return False   
    return True

In [1355]:
def hard_constraint_6(chromosome):
    for i in chromosome.time_table:
        for j in chromosome.time_table:
            if i != j:
                if str(i.invigilator) == str(j.invigilator):
                    if str(i.day) == str(j.day):
                        return False   
    return True
        

In [1356]:
def check_hard_constraints(chromosome):
    #An exam will be scheduled for each course
    if hard_constraint_1(chromosome) == False:
        #print("constraint1")
        chromosome.fitness -= 2
    #A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
    if hard_constraint_2(chromosome) == False:
        #print("constraint2")
        chromosome.fitness -= 2
    #Exam will not be held on weekends.
    if hard_constraint_3(chromosome) == False:
        #print("constraint3")
        chromosome.fitness -= 2
    #Each exam must be held between 9 am and 5 pm
    if hard_constraint_4(chromosome) == False:
        #print("constraint4")
        chromosome.fitness -= 2 
    #Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
    if hard_constraint_5(chromosome) == False:
        #print("constraint5")
        chromosome.fitness -= 2
    #A teacher cannot invigilate two exams in a row.
    if hard_constraint_6(chromosome) == False:
        #print("constraint6")
        chromosome.fitness -= 2   


In [1357]:
def check_soft_constraints(chromosome):
    if soft_constraint_2(chromosome) == False:
        chromosome.fitness -= 1
    #All other soft constraints are already satisfied with our approach

In [1358]:
def check_fitness(chromosome):
    check_hard_constraints(chromosome)
    #check_soft_constraints(chromosome)

In [1359]:
def selection(random1, random2, population_count):
    random1 = random.randint(1,population_count-1)
    random2 = random.randint(1,population_count-1)
    
    return (random1, random2)
    

In [1360]:
def crossover(chromosome1, chromosome2):
    temp = random.randint(0,22)
    #print(temp)
    time_table = list()
    child = Chromosome(time_table, 20)
    for i in range(0,temp):
        child.time_table[i] = chromosome1.time_table[i]
    for i in range(temp, 22):
        child.time_table[i] = chromosome2.time_table[i]
    return child

In [1361]:
def mutation(chromosome):
    temp = random.randint(0,3)
    for i in chromosome.time_table:
        if temp == 0:
            temp2 = random.randint(0,7)
            i.room = rooms[temp2]
        elif temp == 1:
            temp2 = random.randint(0,62)
            i.invigilator = invigilator_names[temp2]
        elif temp == 2:
            temp2 = random.randint(0,29)
            i.day = days[temp2]
        elif temp == 3:
            temp2 = random.randint(0,1)
            i.time = exam_timings[temp2]
    return chromosome

In [1362]:
#create initial population (population of 100 chromosomes)
population_count = 5000
random1 = 0
random2 = 0
initial_population = list()
#creating initial population
for i in range(0, population_count):
    time_table = list()
    initial_population.append(Chromosome(time_table, 20))
#checking fitness
for i in range(0, population_count):
    check_fitness(initial_population[i])
    #print(initial_population[i].fitness)
#sorting population
initial_population.sort(key = lambda x: x.fitness, reverse=True)

#selection and crossover and mutation
#new population
new_population = list()
for i in range(0, population_count):
    #selection
    (random1, random2) = selection(random1, random2, population_count)
    #print(random1, random2)
    #crossover
    child = crossover(initial_population[random1], initial_population[random2])
    #mutation
    child = mutation(child)
    #fitness
    check_fitness(child)
    #appending new child chromosomes
    new_population.append(child)

In [1363]:
#fitness values at each iteration.
for i in range(0, population_count-1):
    print(new_population[i].fitness)

16
16
16
16
18
14
16
16
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
16
16
18
16
16
16
16
16
16
16
14
18
18
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
18
16
14
16
16
16
16
12
12
16
16
16
16
16
16
12
16
16
16
16
16
16
16
12
16
16
16
14
16
16
14
16
16
16
16
16
16
12
16
16
16
14
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
18
16
16
16
16
16
16
16
16
16
14
16
16
12
16
16
16
16
12
14
16
16
12
12
16
16
16
16
16
18
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
16
12
16
12
16
16
16
12
16
16
16
16
16
12
16
16
16
16
16
16
14
16
14
16
16
16
16
16
16
16
14
16
16
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
12
16
16
16
16
12
1

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
14
12
12
16
16
16
16
16
16
16
16
14
16
16
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
12
16
16
16
16
16
16
16
16
16
12
16
16
16
18
16
16
12
16
16
16
16
16
16
16
16
16
16
14
16
16
16
18
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
16
16
16
16
16
16
18
16
16
16
14
16
16
16
16
12
16
16
16
16
16
16
16
16
16
16
12
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
16
16
14
16
16
16
16
16
12
16
14
16
16
16
16
16
16
14
16
16
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
18
16
16
12
16
16
16
16
16
16
16
12
16
16
16
16
16
16
14
16
18
16
16
16
16
14
16
16
16
16
16
14
16
16
16
16
14
16
16
16
16
16
16
12
16
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
12
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
14
16
16
16
16
16
14
16
16
16
16
16
16
16
16
12
16
16
16
16
16
1

In [1397]:

def display_time_table(chromosome):
    print("Course Code    Course Name     Room    Invigilator    Date    Time")
    for i in chromosome.time_table:
        print(i.course_code, " ", i.course_name, "   ", i.room_no, "   ", i.invigilator, "   ", i.day, "   ", i.time)

In [1404]:
max_fitness = 0
for i in range(len(new_population)-1):
    if new_population[i].fitness == 18:
        max_fitness = (i)
        break;
display_time_table(new_population[max_fitness])

Course Code    Course Name     Room    Invigilator    Date    Time
CS217   Object Oriented Programming     C306     Gul e Aisha     Monday3     AM 9:00 - 12:00
EE227   Digital Logic Design     C304     Sidra Khalid     Monday4     PM 2:00 - 5:00
CS211   Discrete Structures     C303     Naveed Ahmad     Wednesday1     AM 9:00 - 12:00
SE110   Intro to Software Engineering     C303     Irum Inayat     Friday2     AM 9:00 - 12:00
CS118   Programming Fundamentals     C305     Noreen Jamil     Friday1     AM 9:00 - 12:00
CS219   Database Systems     C305     Ejaz Ahmed     Tuesday4     PM 2:00 - 5:00
CS220   Operating Systems     C304     Farah Jabeen Awan     Monday2     PM 2:00 - 5:00
CS302   Design & Analysis of Algorithms     C301     Asif Naeem     Friday5     AM 9:00 - 12:00
CY2012   Digital Forensics     C302     Naveed Ahmad     Friday4     PM 2:00 - 5:00
CS307   Computer Networks     C308     Sehrish Hassan     Friday3     PM 2:00 - 5:00
CS328   Software Engineering     C308     Sha